# Modeling Notebook


## 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os 
%config Completer.use_jedi = False

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
os.sys.path.append('../src')

## 2. Load Data

In [2]:
df_raw = pd.read_excel('../data/survey_answers.xlsx', sheet_name='Dados_brutos')
df_raw.head()

,Genero,Idade,Peso,Altura,IMC,Escolaridade,Estado_Civil,Filhos,Atividade_fisica_semana,Cigarros_dia,Horas_Dormidas_dia,Se_Capacitando,Outro_emprego,Ambiente_Trabalho,Categoria_Profissional,Cargo,Tempo_Empresa_anos,Tempo_Funcao_anos,Horas_Trabalhadas_Semana,Horas_Trabalhadas_Dia,Pausas_Programadas_minutos,Trabalho_Entre_Ferias_meses,Recebeu_Treinamento,Como_foi_treinado,Trabalho_Chato,Trabalho_Organizado,Prazos_Apertados,Satisfação_Gerencia,Satisfação_Dificuldades,Tempo_Pe_horas,Tempo_Sentado_horas,Tempo_Corcoras_horas,Superiores_Desconfortavel_horas,Inferiores_Desconfortavel_horas,Tronco_Curvado_horas,Tronco_Torcido_horas,Maos_Dedos_horas,Carga_6_horas,Carga_15_horas,Carga_25_horas,Movimentos_repetitivos_horas,Movimentos_Rapidos_horas,Ferramentas_Maos_horas,Ferramentas_Corpo_horas,Pescoco,Cervical,Costa_superior,Costa_media,Costa_inferior,Bacia,Ombro_esquerdo,Ombro_direito,Braco_esquerdo,Braco_direito,Cotovelo_esquerdo,Cotovelo_direito,Antebraco_esquerdo,Antebraço_direito,Mao_esquerda,Mao_direita,Coxa_esquerda,Coxa_direita,Joelho_esquerdo,Joelho_direito,Perna_esquerda,Perna_direita,Tornozelo_esquerdo,Tornozelo_direito,Pe_esquerdo,Pe_direito,Dedos_direito,Polegar_direito,Interfalange_direito,Palma_mao_direito,Punho_direito,Radial_dist_direito,Ulnar_dist_direito,Radial_prox_direito,Ulnar_prox_direito,Dedos_esquerdo,Polegar_esquerdo,Interfalange_esquerdo,Palma_mao_esquerdo,Punho_esquerdo,Radial_dist_esquerdo,Ulnar_dist_esquerdo,Radial_prox_esquerdo,Ulnar_prox_esquerdo,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,1,36,61.0,1.67,21.872423,3,2,2,0,0,6,2,2,2,1,Tecnico_de_enfermagem,8.0,3.0,36,12.0,30,11,2,1,1,5,5,5,4,11.0,0.5,0.0,0.0,0.0,1.0,0.0,12.0,1.0,1.0,1.0,12.0,12.0,1.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,2,1,5,5,2,5,5,5,5,3,5,5,5,4,5,5,3,5,5,5,4,2,2,5,4,3,4,5,2,2,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,3,3,4,4,4,2,2,3,3,4,3,4,4,3,2,4,4,3,4,2,4,2
1,1,30,65.0,1.62,24.767566,3,2,1,5,0,6,1,2,2,1,Tecnico_de_enfermagem,4.0,7.0,40,8.0,120,11,1,5,1,4,1,4,3,4.0,4.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,8.0,0.0,0.0,0,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,2,1,1,1,2,1,1,1,1,1,1,3,1,4,5,4,1,5,5,5,2,4,5,4,3,4,5,1,5,5,5,5,5,4,2,2,2,1,1,2,4,3,3,2,4,4,2,3,4,4,4,3,5,4,4,2,2,5,2,2,2,5,5,5,2,3,3,2,2,2,3,2,4,5,3,5,5,2,2,2,5
2,1,40,79.0,1.51,34.647603,3,2,3,0,0,10,1,2,1,1,Tecnico_de_enfermagem,8.0,12.0,36,12.0,60,11,1,4,1,5,1,4,3,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,12.0,3.0,0.0,0,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,5,5,5,5,5,5,5,5,2,5,2,4,5,5,2,5,5,5,5,2,2,3,2,5,2,2,3,2,2,3,2,4,4,1,4,4,4,5,4,5,5,5,1,1,5,1,2,2,5,5,5,1,1,3,2,1,5,2,2,1,4,3,4,5,2,2,1,5
3,1,45,76.0,1.60,29.687500,3,2,1,3,0,8,1,2,1,1,Tecnico_de_enfermagem,8.0,18.0,36,12.0,60,11,1,5,2,5,4,4,3,6.0,6.0,0.0,4.0,8.0,2.0,0.0,12.0,0.0,0.0,0.0,12.0,3.0,0.0,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,5,5,5,5,5,5,5,4,3,5,4,4,5,5,3,5,5,5,5,2,2,2,2,1,2,2,3,2,2,2,5,4,4,2,5,4,4,4,5,5,4,4,1,1,1,1,1,1,4,5,4,1,3,4,1,2,3,2,3,2,3,4,4,4,1,1,1,3
4,1,41,67.0,1.68,23.738662,5,1,2,5,0,7,2,2,1,1,Tecnico_de_enfermagem,4.0,4.0,36,12.0,0,11,1,3,1,5,3,4,4,11.0,1.0,1.0,10.0,10.0,0.0,0.0,12.0,1.0,1.0,1.0,12.0,10.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,1,1,1,1,1,2,1,1,5,1,1,1,1,1,2,1,1,5,1,1,1,1,3,1,1,1,5,5,5,5,5,5,5,1,3,3,5,5,4,5,4,5,4,5,5,2,2,4,4,5,4,3,4,2,4,4,4,4,4,2,4,4,5,3,3,3,3,5,3,3,4,2,2,2,3,4,4,2,4,2,3,1,3,3,1,4,3,4,5,5,2,2,3,4


## 3. Feature Engineering

In [3]:
df_analysis = df_raw.copy()
df_analysis['IMC'] = df_analysis['Peso']/df_analysis['Altura']**2

In [4]:
def bmi_category(bmi_value):
    if bmi_value < 18:
        return 'underweight'
    elif bmi_value >= 18 and bmi_value <= 24:
        return 'normal'
    elif bmi_value > 24 and bmi_value <= 28:
        return 'overweight'
    elif bmi_value > 28:
        return 'obese'
    else:
        return 'error'

In [5]:
df_analysis.insert(loc = 1, column = 'IMC_categoria', value = df_analysis['IMC'].apply(bmi_category))

In [6]:
df_analysis.insert(loc = 1, 
                   column = 'Horas_Carregando_Carga', 
                   value = df_analysis['Carga_6_horas'] + df_analysis['Carga_15_horas'] + df_analysis['Carga_25_horas'])

## 4. Target definition

In [7]:
def target_grouping(score):
    if score >= 4:
        return 'severe pain'
    elif score in [2,3]:
        return 'mild pain'
    else:
        return 'no pain'

In [8]:
df_analysis['Costa_superior_agg'] = df_analysis['Costa_superior'].apply(target_grouping)
df_analysis['Costa_media_agg'] = df_analysis['Costa_media'].apply(target_grouping)
df_analysis['Costa_inferior_agg'] = df_analysis['Costa_inferior'].apply(target_grouping)

## 5. Feature Selection

In [9]:
continous_columns = [
                    'Idade', 
                    'Horas_Trabalhadas_Dia',
                    'Tempo_Empresa_anos',
                    'Trabalho_Entre_Ferias_meses',
                    'Tempo_Pe_horas',
                    'Tempo_Sentado_horas',
                    'Tempo_Corcoras_horas',
                    'Cigarros_dia',
                    'Horas_Dormidas_dia',
                    'Pausas_Programadas_minutos',
                    'Superiores_Desconfortavel_horas',
                    'Inferiores_Desconfortavel_horas',
                    'Tronco_Curvado_horas',
                    'Tronco_Torcido_horas',
                    'Maos_Dedos_horas',
                    'Carga_6_horas',
                    'Carga_15_horas',
                    'Carga_25_horas',
                    'Movimentos_repetitivos_horas',
                    'Movimentos_Rapidos_horas',
                    'Ferramentas_Maos_horas',
                    'Ferramentas_Corpo_horas',
                    ]
categorical_columns = ['Genero']
target_columns = ['Costa_superior_agg', 'Costa_media_agg', 'Costa_inferior_agg']

useful_columns = continous_columns + categorical_columns + target_columns

## 6. Train and test split
We'll split the data into train and test set. The train set will be used to perform 10-fold cross validation. We'll compute metrics for the test set and compare them.

In [10]:
numerical_features = [
    'Idade', 
    'Filhos',
    'Horas_Trabalhadas_Semana',
    'Tempo_Empresa_anos',
    'Trabalho_Entre_Ferias_meses',
    'Tempo_Pe_horas',
    'Tempo_Sentado_horas',
    'Tempo_Corcoras_horas',
    'Cigarros_dia',
    'Horas_Dormidas_dia',
    'Pausas_Programadas_minutos',
    'Superiores_Desconfortavel_horas',
    'Inferiores_Desconfortavel_horas',
    'Tronco_Curvado_horas',
    'Tronco_Torcido_horas',
    'Maos_Dedos_horas',
    'Carga_6_horas',
    'Carga_15_horas',
    'Carga_25_horas',
    'Movimentos_repetitivos_horas',
    'Movimentos_Rapidos_horas',
    'Ferramentas_Maos_horas',
    'Ferramentas_Corpo_horas',
    'Horas_Carregando_Carga'
]

categorical_features = [
    'IMC_categoria'    
]

In [11]:
from sklearn.model_selection import train_test_split

X = df_analysis[numerical_features + categorical_features]
y = df_analysis[target_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [12]:
# scorer = make_scorer(accuracy_score) # Teste outras

# cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
# gscv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

# algorithms = {
#     'kNN':  GridSearchCV(
#         Pipeline([
#             ('imputer', SimpleImputer(strategy='mean')),
#             ('scaler', MinMaxScaler(feature_range=(0, 1))),
#             ('selector', VarianceThreshold()),
#             ('knn', KNeighborsClassifier())]), 
#         param_grid={
#             'selector__threshold': [0, 0.01, 0.02, 0.03],
#             'knn__n_neighbors': [1, 3, 5],
#             'knn__p': [1, 2],
#         },
#         scoring=scorer,
#         cv=gscv),
# }

import experiments

algorithms = experiments.algorithms
cv = experiments.cv

In [13]:
from sklearn.model_selection import cross_val_score

result = {}
for alg, clf in algorithms.items():
    result[alg] = cross_val_score(clf, X_train, y_train['Costa_superior_agg'], cv=cv)

result_df = pd.DataFrame.from_dict(result)

/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/Users/56325/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too 

In [15]:
result_df

,kNN,kNN_2nd,nb_1st,nb_2nd
0,0.433333,0.433333,0.333333,0.166667
1,0.500000,0.500000,0.500000,0.433333
2,0.566667,0.566667,0.366667,0.366667
3,0.500000,0.533333,0.200000,0.233333
4,0.482759,0.448276,0.379310,0.241379
5,0.482759,0.379310,0.275862,NaN
6,0.517241,0.482759,0.206897,0.448276
7,0.379310,0.517241,0.206897,0.379310
8,0.482759,0.448276,0.275862,0.275862
9,0.517241,0.586207,0.241379,0.275862
